In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

# Disposals Outputs

#### Child counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
   Disp_type AS Disposal_type,
   order_desc AS Order_type,
   order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_disps_all_children
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End,
  Disp_type,
  order_desc,
  order_code
  
""",

"child_counts")

In [ ]:
pydb.read_sql_query ("select * from __temp__.child_counts where disposal_type <> 'Order'")

### Order counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Order type' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
   Disp_type AS Disposal_type,
   order_desc AS Order_type,
   order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_disp_order_type
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End,
  Disp_type,
  order_desc,
  order_code
  
""",

"order_type_counts")

In [ ]:
pydb.read_sql_query ("select * from __temp__.order_type_counts where disposal_type = 'Order'")

### Disposal events

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Disposal events' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'N/A' AS Disposal_type,
   'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_disp_events
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End
  
""",

"disp_event_counts")

In [ ]:
pydb.read_sql_query ("select * from __temp__.disp_event_counts where year > 2019")

### Case counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Cases' AS Type,
  'Cases closed' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'N/A' AS Disposal_type,
   'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_closed_cases
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End
  
""",

"disp_case_counts")

In [ ]:
pydb.read_sql_query ("select * from __temp__.disp_case_counts")

In [ ]:
pydb.read_sql_query (f"""select * from {fcsq_db}.ca_order_type_lookup""")

### Combine all datasets

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  *
FROM
  __temp__.child_counts
UNION ALL
SELECT
  *
FROM
  __temp__.order_type_counts
UNION ALL
SELECT
  *
FROM
  __temp__.disp_event_counts
UNION ALL 
SELECT
  *
FROM
  __temp__.disp_case_counts

  
""",

"disp_csv")

In [ ]:
data = pydb.read_sql_query ("select * from __temp__.disp_csv")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv (r's3://alpha-family-data/fcsq_processing/CA_disps/test_csv/ca_disps.csv', header = True)